## DSC 550-T302
## Week 2 Assignment
## Lincoln Brown
## Professor Werner

## Adult or Census Income Dataset
### https://archive.ics.uci.edu/dataset/2/adult

For the assignment this week, we will be working with the adult dataset also known as the Census Income Dataset. 
This dataset was extracted from the 1994 Census database by Barry Becker. 

This 



In [161]:
# Import libraries 
import numpy as np
import pandas as pd
import re
import seaborn as sns

In [162]:
# Load adult.names file to extract column names
names_file = "adult.names"
with open(names_file, 'r') as data:
    names_data = data.readlines()

In [163]:
# The column names are contained in the last 15 lines of the text file
# Each column name ends with a colon, some of them also contain hyphens
# We will use regex to extract based on this pattern
col_names = []
#pattern = re.compile(r'\w+(?:-\w+)?:')
pattern = re.compile(r'\w+(?:-\w+)?(?:-\w+)?:')
for line in names_data[-15:]:
    col_name = pattern.findall(line)
    if len(col_name) > 0:
        col_names.append(str(col_name[0]).strip(':'))

print(col_names)

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


In [166]:
# Load the adult.data dataframe using the pd.read_csv function.
# We will also specify the headers at this time
data_file = "adult.data"
#adult_df = pd.read_csv(data_file, header=None)
adult_df = pd.read_csv(data_file, header=None)
adult_df.columns = col_names
adult_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
